In [31]:
# Universaidad Nacional Autónoma de México
# Facultad de Ingeniería
# Aprendizaje (Máquina)
# Programa 04
# Algoritmo Genético
# Aguilar Enriquez Paul Sebastian
# Cabrera Lopez Oscar Emilio

In [32]:
import random

In [33]:
# Funciones de apoyo

## Verifica la suma de capital a invertir y que este se encuentre en el 
## rango valido
def inversionValida(a, b, c, d):
    inversion = a + b + c + d
    if (inversion > 0) and (inversion <= 10):
        return True
    else:
        return False
    
## Convierte de decimal a binario
def dec2bin(dec):
    bina = []
    
    # Convertimos el valor decimal a binario
    while int(dec/2) != 0:
        bina.append( int(dec%2) )
        #print(bina)
        dec = dec/2
        #print(dec)
    if dec > 0:
        bina.append(1)
        #print(bina)
    
    # Completamos los valores faltantes
    while(len(bina) < 4):
        bina.append(0)
    #print(bina)
    bina.reverse()
    return bina

In [34]:
# Metodo de selección: -
# Cruza: De 2 puntos , crossover_rate(pc) = 80%
crossover_rate = .8
# Mutación: -, mutation_rate(pm) = 1%
mutation_rate = .01
# Tamaño de población: 50
population = 50
# Cantidad de generaciones: 20
generations = 20

In [35]:
# Función de aptitud
def F(T1, T2, T3, T4):
    sumInversionTotal = T1 + T2 + T3 + T4;
    V = abs(sumInversionTotal - 10)
    
    # Ahora sí función de aptitud :D
    x = sumInversionTotal / ( (500*V)+1 )
    return x

In [36]:
# Generador de población (Inicialización)
pobDec = []
pobBin = []

def generador():
    # Creamos la cantidad de poblacion
    for i in range(population):
        # Limpiamos valores
        a = 0
        b = 0
        c = 0
        d = 0
        
        # Proponemos los valores a invertir de manera aleatoria
        # y verificamos que se encuentren en el rango de 
        # 0 <= inversion <= 10
        while(not inversionValida(a, b, c, d)):
            a = random.randint(0, 11)
            b = random.randint(0, 11 - a)
            c = random.randint(0, 11 - a - b)
            d = random.randint(0, 11 - a - b - c)
        
        # Guardamos al individuo en su formato decimal
        pobDec.append([a, b, c, d])
        # Guardamos al individuo en su formato binario
        pobBin.append([dec2bin(a), dec2bin(b), dec2bin(c), dec2bin(d)])

# Imprime la poblacion feliz de nuestro universo :D
def poblacion():
    print("# Población inicial #")
    for i in range(population):
        print("Individuo " , i, ": ", pobDec[i])
        print("Binario: ", i, ": ", pobBin[i])

In [37]:
def main():
    generador()
    poblacion()

In [38]:
main()

# Población inicial #
Individuo  0 :  [1, 1, 1, 3]
Binario:  0 :  [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 1]]
Individuo  1 :  [1, 2, 3, 4]
Binario:  1 :  [[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 1], [0, 1, 0, 0]]
Individuo  2 :  [4, 5, 1, 0]
Binario:  2 :  [[0, 1, 0, 0], [0, 1, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]]
Individuo  3 :  [8, 0, 2, 0]
Binario:  3 :  [[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0]]
Individuo  4 :  [3, 6, 1, 0]
Binario:  4 :  [[0, 0, 1, 1], [0, 1, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0]]
Individuo  5 :  [4, 1, 5, 0]
Binario:  5 :  [[0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 1], [0, 0, 0, 0]]
Individuo  6 :  [3, 0, 5, 2]
Binario:  6 :  [[0, 0, 1, 1], [0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 1, 0]]
Individuo  7 :  [4, 5, 1, 0]
Binario:  7 :  [[0, 1, 0, 0], [0, 1, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]]
Individuo  8 :  [7, 1, 1, 0]
Binario:  8 :  [[0, 1, 1, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 0]]
Individuo  9 :  [2, 2, 4, 1]
Binario:  9 :  [[0, 0, 1, 0], [0, 0, 1, 